In [1]:
import pandas as pd
import numpy as np
import os
import glob
import re
import random
import pickle

In [2]:
# read in all flights
root = os.getcwd()
data_dir = os.path.join(root, 'instance1')

dfs = []
for file in os.listdir(data_dir):
    if file.startswith('day') and file.endswith('.csv'):
        dfs.append(pd.read_csv(os.path.join(data_dir, file)))
    
df = pd.concat(dfs)
df.columns = df.columns.str.strip()
for col in df.select_dtypes(include=["object"]):
    df[col] = df[col].str.strip()
    
df["#leg_nb"] = df["#leg_nb"].astype(str)
df = df.set_index('#leg_nb')
df["dep_datetime"] = pd.to_datetime(df["date_dep"] + " " + df["hour_dep"])
df["arr_datetime"] = pd.to_datetime(df["date_arr"] + " " + df["hour_arr"])
df['duration'] = (df['arr_datetime'] - df['dep_datetime']).dt.total_seconds() / 3600

In [3]:
def parse_initial_solution_base2(path):
    pairings = {}

    with open(path, "r") as f:
        text = f.read()

    # Clean wrapper
    text = text.replace("Solution = {", "").replace("};", "").strip()

    # Split into pairing id + content blocks
    raw_entries = re.split(r'\s*Pairing\s+(\d+)\s*:', text)[1:]

    # Loop through [id, content, id, content, ...]
    for i in range(0, len(raw_entries), 2):
        pairing_id = int(raw_entries[i])
        content = raw_entries[i+1].strip()

        # Extract base
        base_match = re.search(r'Base\s+(BASE\d+)\s*:', content)
        base = base_match.group(1) if base_match else None

        # Requirement (1) — only include BASE2
        if base != "BASE2":
            continue

        # Extract legs (including TDH)
        legs = re.findall(r'(?:TDH_)?LEG_\d+_\d+', content)

        # Store in dictionary
        pairings[pairing_id] = {
            "base": base,
            "legs": legs
        }

    return pairings

pairing_dict = parse_initial_solution_base2("instance1/initialSolution.in")

In [9]:
count = 0

for pairing in pairing_dict:
    flights = pairing_dict[pairing]
    count += len(flights["legs"])

count

684

In [4]:
def count_overnights(legs, airlegs_df):
    # Select rows by index directly
    df = airlegs_df.loc[legs].copy()

    # Sort chronologically
    df = df.sort_values('dep_datetime')

    # Overnight = next dep date > current arr date
    overnights = (
        df['dep_datetime'].shift(-1).dt.date >
        df['arr_datetime'].dt.date
    ).sum()

    return int(overnights)

In [5]:
for idx, pairing in pairing_dict.items():
    pairing_dict[idx]["length"] = len(pairing['legs'])
    # compute duration
    pairing_legs = pairing['legs']
    clean_legs = [leg.replace("TDH_", "") for leg in pairing_legs]
    pairing_dict[idx]["flight_time"] = sum([df.loc[leg]["duration"] for leg in clean_legs])
    pairing_dict[idx]["overnights"] = count_overnights(clean_legs, df)


In [6]:
def compute_pairing_spans(pairings_dict, legs_df):
    spans = {}

    for pid, pinfo in pairings_dict.items():
        # Normalize leg IDs (remove TDH_)
        clean_legs = [leg.replace("TDH_", "") for leg in pinfo["legs"]]

        # Compute span
        start = min(legs_df.loc[leg, "dep_datetime"] for leg in clean_legs)
        end   = max(legs_df.loc[leg, "arr_datetime"] for leg in clean_legs)

        spans[pid] = (start, end)
        pairing_dict[pid]["start"] = start
        pairing_dict[pid]["end"] = end

    return spans

In [14]:
with open('pairings.pickle', 'wb') as f:
    pickle.dump(pairing_dict, f)

In [7]:
spans = compute_pairing_spans(pairing_dict, df)

In [8]:
with open('spans.pickle', 'wb') as f:
    pickle.dump(spans, f)

In [8]:
import numpy as np

def compute_weights(pairing_dict, alpha=1, beta=1):
    pids = list(pairing_dict)
    lengths = np.array([pairing_dict[p]["length"] for p in pids], float)
    flights = np.array([pairing_dict[p]["flight_time"] for p in pids], float)

    # normalize
    L = (lengths - lengths.mean()) / (lengths.std() + 1e-6)
    F = (flights - flights.mean()) / (flights.std() + 1e-6)

    weights = {pids[i]: alpha*L[i] + beta*F[i] for i in range(len(pids))}
    return weights


In [9]:
def make_buckets(pids, weights):
    p_sorted = sorted(pids, key=lambda p: weights[p], reverse=True)
    b1, b2, b3 = [], [], []
    for i, p in enumerate(p_sorted):
        if i % 3 == 0: b1.append(p)
        elif i % 3 == 1: b2.append(p)
        else: b3.append(p)
    b3 = b3[::-1]   # reverse bucket 3 for better balancing
    return b1, b2, b3


In [10]:
def overlaps(p, q, spans):
    s1, e1 = spans[p]
    s2, e2 = spans[q]
    return not (e1 <= s2 or e2 <= s1)


In [11]:
def triple_valid(p1, p2, p3, spans):
    return (
        not overlaps(p1,p2,spans) and
        not overlaps(p1,p3,spans) and
        not overlaps(p2,p3,spans)
    )


In [12]:
def order_by_start(triple, spans):
    return tuple(sorted(triple, key=lambda p: spans[p][0]))


In [13]:
def pack_balanced_nonoverlapping_triples(pairing_dict, spans, alpha=1, beta=1):
    # compute weights
    weights = compute_weights(pairing_dict, alpha, beta)
    pids = list(pairing_dict)
    
    # sort & bucket
    b1, b2, b3 = make_buckets(pids, weights)
    
    triples = []
    used = set()
    
    # greedy balanced triple pairing
    for p1, p2, p3 in zip(b1, b2, b3):
        if p1 in used or p2 in used or p3 in used:
            continue
        if triple_valid(p1, p2, p3, spans):
            t = (p1, p2, p3)
            t = order_by_start(t, spans)
            triples.append(t)
            used.update([p1, p2, p3])
    
    return triples


In [ ]:
# Improved triple-packing heuristics
# 1) Candidate-based greedy: generate all valid triples from buckets, score by balance & early starts, and greedily pick non-overlapping unique triples.
# 2) Randomized repeated greedy: run the greedy heuristic with random shuffles/weights multiple times and return the best solution.

from itertools import product


def triple_score(triple, weights, spans):
    # Score to prefer balanced weight triples and early-ending triples
    w = [weights[p] for p in triple]
    balance_penalty = abs(w[0]-w[1]) + abs(w[0]-w[2]) + abs(w[1]-w[2])
    # encourage earlier end-times (more spare room for others)
    end_times = [spans[p][1] for p in triple]
    end_penalty = sum([(et - min(end_times)).total_seconds() for et in end_times])
    # negative because lower penalty -> higher score
    return -balance_penalty - 1e-6 * end_penalty


def pack_balanced_nonoverlapping_triples_candidate(pairing_dict, spans, alpha=1, beta=1):
    """Generate all valid triples across 3 buckets, score them, and greedily select disjoint triples.

    This algorithm is O(|b1||b2||b3|) worst-case and finds a better packing than the zip-based approach.
    """
    weights = compute_weights(pairing_dict, alpha, beta)
    pids = list(pairing_dict)
    b1, b2, b3 = make_buckets(pids, weights)

    # Build candidate triples
    candidates = []
    # We'll iterate b1 x b2 x b3, but with pruning when pair overlaps
    for p1 in b1:
        for p2 in b2:
            if overlaps(p1, p2, spans):
                continue
            for p3 in b3:
                if overlaps(p1, p3, spans) or overlaps(p2, p3, spans):
                    continue
                t = (p1, p2, p3)
                score = triple_score(t, weights, spans)
                candidates.append((score, t))

    # Sort by score descending and greedily pick disjoint triples
    candidates.sort(reverse=True, key=lambda s_t: s_t[0])

    used = set()
    triples = []
    for score, t in candidates:
        if t[0] in used or t[1] in used or t[2] in used:
            continue
        triples.append(order_by_start(t, spans))
        used.update(t)

    return triples


def pack_balanced_nonoverlapping_triples_randomized(pairing_dict, spans, alpha=1, beta=1, iterations=500, seed=None):
    """Randomized greedy: shuffle buckets and run greedy candidate-based selection multiple times to find best solution.

    This often helps escape local optima from deterministic ordering.
    """
    if seed is not None:
        random.seed(seed)

    best = []
    weights = compute_weights(pairing_dict, alpha, beta)
    pids = list(pairing_dict)

    for it in range(iterations):
        # shuffle slight ordering inside each bucket to diversify candidate generation
        b1, b2, b3 = make_buckets(pids, weights)
        # introduce tiny random perturbation in order to break ties
        random.shuffle(b1)
        random.shuffle(b2)
        random.shuffle(b3)

        # Build candidate triples with the shuffled buckets; we will stop early if no more
        candidates = []
        for p1 in b1:
            for p2 in b2:
                if overlaps(p1, p2, spans):
                    continue
                for p3 in b3:
                    if overlaps(p1, p3, spans) or overlaps(p2, p3, spans):
                        continue
                    t = (p1, p2, p3)
                    score = triple_score(t, weights, spans) + random.random() * 1e-3
                    candidates.append((score, t))

        random.shuffle(candidates)
        candidates.sort(reverse=True, key=lambda s_t: s_t[0])

        used = set()
        triples = []
        for score, t in candidates:
            if t[0] in used or t[1] in used or t[2] in used:
                continue
            triples.append(order_by_start(t, spans))
            used.update(t)

        if len(triples) > len(best):
            best = triples

    return best


In [14]:
triples = pack_balanced_nonoverlapping_triples(pairing_dict, spans)

In [ ]:
# Test improved algorithms and compare counts

orig_triples = pack_balanced_nonoverlapping_triples(pairing_dict, spans)
print('Original greedy zip() count:', len(orig_triples))

cand_triples = pack_balanced_nonoverlapping_triples_candidate(pairing_dict, spans)
print('Candidate greedy count:', len(cand_triples))

rand_triples = pack_balanced_nonoverlapping_triples_randomized(pairing_dict, spans, iterations=200, seed=1234)
print('Randomized greedy best count (200 iters):', len(rand_triples))

# Show a few sample triples if available
for name, tset in [('orig', orig_triples), ('cand', cand_triples), ('rand', rand_triples)]:
    print('\n', name.upper(), 'triples sample (first 5):')
    for t in tset[:5]:
        wsum = sum(compute_weights(pairing_dict)[p] for p in t)
        print(t, 'weight-sum=%.3f' % wsum)

In [15]:
def assert_no_airleg_overlap(legs, airlegs_df):
    """
    Raises an error if ANY air legs in `legs` overlap in time.
    
    Parameters
    ----------
    legs : list of leg IDs (strings)
    airlegs_df : pandas DataFrame indexed by leg_id with columns:
        - dep_datetime (datetime64)
        - arr_datetime (datetime64)
    """
    
    # Extract only the relevant legs
    df = airlegs_df.loc[legs].copy()
    
    # Sort by departure time
    df = df.sort_values("dep_datetime")
    
    # Iterate pairwise
    for i in range(len(df) - 1):
        current_end = df.iloc[i]["arr_datetime"]
        next_start = df.iloc[i+1]["dep_datetime"]
        
        if current_end > next_start:
            raise ValueError(
                f"Overlap detected between legs "
                f"{df.index[i]} (ends {current_end}) and "
                f"{df.index[i+1]} (starts {next_start})."
            )
    
    # If no overlap:
    return True


In [16]:
lines = {}
for idx, triple in enumerate(triples):
    lines[idx] = {}
    legs = []
    flight_time = 0
    overnights = 0
    for pairing in triple:
        pairing_legs = pairing_dict[pairing]["legs"]
        clean_legs = [leg.replace("TDH_", "") for leg in pairing_legs]
        legs += clean_legs
        flight_time += pairing_dict[pairing]["flight_time"]
        overnights += pairing_dict[pairing]["overnights"]
    assert assert_no_airleg_overlap(legs, df)
    lines[idx]["legs"] = legs
    lines[idx]["num_legs"] = len(legs)
    lines[idx]["overnights"] = overnights
    lines[idx]["flight_time"] = flight_time

In [17]:
lines

{0: {'legs': ['LEG_01_21',
   'LEG_01_4',
   'LEG_01_32',
   'LEG_02_10',
   'LEG_02_11',
   'LEG_02_19',
   'LEG_02_23',
   'LEG_03_22',
   'LEG_03_14',
   'LEG_04_35',
   'LEG_11_29',
   'LEG_11_24',
   'LEG_26_24',
   'LEG_26_25',
   'LEG_26_28',
   'LEG_27_14',
   'LEG_27_10',
   'LEG_27_17',
   'LEG_27_21',
   'LEG_28_20',
   'LEG_28_14',
   'LEG_28_10'],
  'num_legs': 22,
  'overnights': 2,
  'flight_time': np.float64(44.300000000000004)},
 1: {'legs': ['LEG_03_3',
   'LEG_03_2',
   'LEG_14_27',
   'LEG_14_17',
   'LEG_14_18',
   'LEG_15_14',
   'LEG_15_13',
   'LEG_15_22',
   'LEG_16_20',
   'LEG_16_15',
   'LEG_16_16',
   'LEG_16_6',
   'LEG_26_26',
   'LEG_26_3',
   'LEG_27_1',
   'LEG_27_2',
   'LEG_27_7',
   'LEG_28_21',
   'LEG_28_23',
   'LEG_28_5',
   'LEG_28_29',
   'LEG_29_26'],
  'num_legs': 22,
  'overnights': 3,
  'flight_time': np.float64(41.8)},
 2: {'legs': ['LEG_08_14',
   'LEG_09_35',
   'LEG_10_1',
   'LEG_10_34',
   'LEG_10_33',
   'LEG_11_10',
   'LEG_11_11',

In [18]:
with open("lines.pickle", "wb") as f: # 'wb' for write binary
    pickle.dump(lines, f)